In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import sampler
import torchvision.models as models


In [3]:
dic = torch.load("/home/drc/Downloads/cifar10-d875770b.pth")

In [4]:
model =torch.utils.model_zoo.load_url('https://download.pytorch.org/models/vgg11-bbd30ac9.pth')

In [3]:
model.keys()

[u'features.0.weight',
 u'features.0.bias',
 u'features.3.weight',
 u'features.3.bias',
 u'features.6.weight',
 u'features.6.bias',
 u'features.8.weight',
 u'features.8.bias',
 u'features.11.weight',
 u'features.11.bias',
 u'features.13.weight',
 u'features.13.bias',
 u'features.16.weight',
 u'features.16.bias',
 u'features.18.weight',
 u'features.18.bias',
 u'classifier.0.weight',
 u'classifier.0.bias',
 u'classifier.3.weight',
 u'classifier.3.bias',
 u'classifier.6.weight',
 u'classifier.6.bias']

In [ ]:
#attempt to prune filters using taylor criterion


class PruneWrap():
    def __init__(self, model,gpu=False):
        #model params
        self.model = model
        self.gpu = gpu
        self.criterion = torch.nn.CrossEntropyLoss()
        #bookkeeping
        self.activations = []
        self.gradients = []
        self.grad_index = 0
        self.activation_to_layer = {}
        self.filter_ranks = {}
        #callbacks for gradient updates
        register_callbacks()
    def register_callbacks(self):
        activation_index = 0
        for layer, (name, module) in enumerate(self.model.features._modules.items()):
            x = module(x)
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                x.register_hook(self.compute_rank)
                self.activations.append(x)
                self.activation_to_layer[activation_index] = layer
                activation_index += 1
    def test(self,test_data_loader):
        self.model.eval()
        correct = 0
        total = 0
        for i, (batch, label) in enumerate(test_data_loader):
            batch = batch.cuda() if gpu else batch
            output = model(Variable(batch))
            pred = output.data.max(1)[1]
            correct += pred.cpu().eq(label).sum()
            total += label.size(0)
        print "acc :", float(correct) / total
    
    def train(self, optimizer = None, epoches = 10, prune=False):
        if optimizer is None:
            optimizer = optim.SGD(model.classifier.parameters(), lr=0.0001, momentum=0.9)
        for i in range(epoches):
            print "epoch: ", i
            for batch, label in self.train_data_loader:
                batch,label = (batch.cuda(),label.cuda) if self.gpu
                self.model.zero_grad()
                input = Variable(batch)
                if prune:
                    output = self.prunner.forward(input)
                    self.criterion(output, Variable(label)).backward()
                else:
                    self.criterion(self.model(input), Variable(label)).backward()
                    optimizer.step()
        print "Finished fine tuning."
    def rank_prunes(self):
        for batch, label in self.train_data_loader:
                batch,label = (batch.cuda(),label.cuda) if self.gpu
                self.model.zero_grad()
                input = Variable(batch)
        
        self.prunner.normalize_ranks_per_layer()

        return self.prunner.get_prunning_plan(num_filters_to_prune)
    def prune():
        self.test()
    def normalize_ranks_per_layer(self):
        for i in self.filter_ranks:
            v = torch.abs(self.filter_ranks[i])
            v = v / np.sqrt(torch.sum(v * v))
    def compute_rank(self, grad):
        activation_index = len(self.activations) - self.grad_index - 1
        activation = self.activations[activation_index]
        values = torch.sum((activation * grad), dim = 0).sum(dim=2).sum(dim=3)[0, :, 0, 0].data

        # Normalize the rank by the filter dimensions
        values = values / (activation.size(0) * activation.size(2) * activation.size(3))

        if activation_index not in self.filter_ranks:
            self.filter_ranks[activation_index] = torch.FloatTensor(activation.size(1)).zero_().cuda()

        self.filter_ranks[activation_index] += values
        self.grad_index += 1


In [ ]:
#attempt to prune for sparsity without wrapper thing
class PruneWrapSparse():
    def __init__(self,epoch_start,model,sparsity_initial,sparsity_target,prune_steps,update_rate,train,gpu=False):
        #model params
        self.model = model.float()
        self.gpu = gpu
        self.criterion = torch.nn.CrossEntropyLoss()
        self.train_loader = train
        self.layers_to_prune = [0,0,0]
        
        #bookkeeping
        self.current_sparsity=0
        self.sparsity_target = sparsity_target
        self.sparsity_initial = sparsity_initial
        self.train_step = epoch_start
        self.prune_steps =prune_steps
        self.update_rate = update_rate
        self.prunning = True
        self.debug=.90
        
    def to_string(self):
        print "current sparsity: " + str(self.current_sparsity)
        print "sparsity target: "+ str(self.sparsity_target)
        print "initial sparsity: " + str(self.sparsity_initial)
        print "current train step: "+ str(self.train_step)
        print "prune steps: "+str(self.prune_steps)
        print "prunning rate: "+str(self.update_rate)
        
    def test(self,test_data_loader):
        self.model.eval()
        correct = 0
        total = 0
        for i, (batch, label) in enumerate(test_data_loader):
            batch,label = (batch.cuda(),label.cuda()) if self.gpu else (batch,label)
            output = self.model(Variable(batch.float()))
            pred = output.data.max(1)[1]
            correct += pred.cpu().eq(label).sum()
            total += label.size(0)
        print "acc :", float(correct) / total
    
    def reset(self):
        self.current_sparsity=0
        self.prune_step =0
    
    def compute_current_target(self):
        return self.sparsity_target+(self.sparsity_initial-self.sparsity_target)*(1-self.train_step/(self.prune_steps*self.train_step))**3
    
    def train(self, optimizer = None, epoches = 10):
        start = time.time()
        if optimizer is None:
            optimizer = optim.SGD(self.model.classifier.parameters(), lr=0.0001, momentum=0.9)
        for i in range(epoches):
            print "epoch: ", i
            for batch, label in self.train_loader:
                if self.current_sparsity >= self.sparsity_target:
                    self.prunning = False
                batch,label = (batch.cuda(),label.cuda()) if self.gpu else (batch,label)
                input = Variable(batch)
                if self.prunning and self.train_step%self.update_rate==0:
                    print "prunning"
                    self.prune()
                    return
                self.criterion(self.model(input), Variable(label)).backward()
                optimizer.step()
                self.train_step+=1
        print "Finished fine tuning."
        print "Time elapsed:" + (time.time() - start)
    
    def prune(self):#all conv layers for now
        current_sparsity_target = self.compute_current_target()
        current_sparsity_target = self.debug  
        for layer in list(self.model.children())[0]:
            if isinstance(layer, nn.modules.conv.Conv2d) or isinstance(layer, nn.modules.Linear):
                #find weight threshold for layer
                weight_threshold = self.get_weight_threshold(layer.parameters(),current_sparsity_target) 
                print(list(layer.parameters()))
                #prune layer
                print()
                self.save_weight_hist(list(layer.parameters())[0])
                print weight_threshold,"here" 
                self.prune_layer(layer, weight_threshold)
                self.save_weight_hist(list(layer.parameters())[0])
            break
        self.current_sparsity = current_sparsity_target
        self.debug+=.02
        
    def prune_layer(self,layer,threshold):
        dim = layer.weight.data.size()
        for i in range(dim[0]):
            for j in range(dim[1]):
                for k in range(dim[2]):
                    for l in range(dim[3]):
                        #if layer.weight.data[i,j,k,l]<threshold:
                            layer.weight.data[i,j,k,l]=0
                            layer.weight[i,j,k,l].detach()
    def save_weight_hist(self,params):
        plt.figure()
        plt.hist(list(params.cpu().data.abs().numpy().flatten()))
        plt.show()
    def get_weight_threshold(self,params,percent_to_prune):
        weights=[]
        for param in params:
            if len(list(param.cpu().data.abs().numpy().flatten()))>1:
                weights.extend(list(param.cpu().data.abs().numpy().flatten()))
        threshold = np.percentile(np.array(weights),percent_to_prune)
        print len(np.nonzero(np.array(weights)))/float(len(weights))
        return threshold
            